# Generate initial min-snap trajectory

In [ ]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import os, sys, time, copy, argparse
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, '../')
from pyTrajectoryUtils.pyTrajectoryUtils.utils import *
from mfboTrajectory.utils import *
from mfboTrajectory.minSnapTrajectoryWaypoints import MinSnapTrajectoryWaypoints

traj_tool = TrajectoryTools(MAX_POLY_DEG = 9, MAX_SYS_DEG = 4, N_POINTS = 20)
min_snap = MinSnapTrajectoryWaypoints(MAX_POLY_DEG = 9, MAX_SYS_DEG = 4, N_POINTS = 20)

# Choose waypoints (check waypoints folder)
points, t_set = get_waypoints('../constraints_data/waypoints_constraints.yaml', 'traj_1', flag_t_set=True)

########################################################
# Min snap trajectory
# Input
#   alpha_scale - scale overall time with alpha_scale value
#   yaw_mode - 0: set all yaw to 0, 1: set trajectory yaw forward, 2: set trajectory yaw as the reference yaw (points[:,3])
#   flag_loop - whether the trajectory has same initial and final position and attitude
#   deg_init_min, deg_init_max - the initial state is set to 0 from deg_init_min - th derivative to deg_init_max - th derivative
#       (e.g. deg_init_min=0, deg_init_max=2: initial position is set to points[0,:] and initial velocity, acceleration is set to zero)
#   deg_end_min, deg_end_max - the final state is set to 0 from deg_end_min - th derivative to deg_end_max - th derivative
#   ded_init_yaw_min=0, deg_init_yaw_max - set the initial yaw value and it's derivative to zero
#   ded_end_yaw_min=0, deg_end_yaw_max - set the fianl yaw value and it's derivative to zero
#   flag_rand_init - whether to use random initialization on optimization process. 
#       (randomly generate set of time allocation and select the on minimze the overall snap)
#   flag_numpy_opt - whether to use numpy-based gradient descent to find initial value (often improve the final result).
# Output
#   t_set - time allocation between waypoints
#   d_ordered - position and derivatives on each waypoint
#   d_ordered_yaw - yaw value and angular speed, acceleration on each waypoint
t_set, d_ordered, d_ordered_yaw = min_snap.get_min_snap_traj(
    points, alpha_scale=1.0, flag_loop=False, yaw_mode=2, \
    deg_init_min=0, deg_init_max=4, \
    deg_end_min=0, deg_end_max=2, \
    deg_init_yaw_min=0, deg_init_yaw_max=2, \
    deg_end_yaw_min=0, deg_end_yaw_max=2, \
    flag_rand_init=False, flag_numpy_opt=False)
print("t_set = np.array([{}])".format(','.join([str(x) for x in t_set])))
print("########################################################")

########################################################
# Change trajectory speed
# multiply alpha_set with t_set and generate new d_orderd (derivates on each waypoints) and d_ordered_yaw
# Check pyTrajectoryUtils/MinSnapTrajectoryUtils class for the details
# Set yaw_mode - 0: set all yaw to 0, 1: set trajectory yaw forward, 2: set trajectory yaw as the reference yaw (points[:,3])
t_set_new, d_ordered, d_ordered_yaw = min_snap.update_traj(
    points=points, \
    t_set=t_set, \
    alpha_set=2.0*np.ones_like(t_set), \
    yaw_mode=2, \
    flag_run_sim=False)
print("t_set_new = np.array([{}])".format(','.join([str(x) for x in t_set_new])))
print("########################################################")

########################################################
# Add rampIn trajectory
# t_set_new, d_ordered, d_ordered_yaw = min_snap.append_rampin(t_set_new, d_ordered, d_ordered_yaw)

########################################################
# Run simulation with indi controller
debug_array = min_snap.sim.run_simulation_from_der( \
    t_set=t_set_new, d_ordered=d_ordered, d_ordered_yaw=d_ordered_yaw, \
    max_pos_err=0.2, min_pos_err=0.1, max_yaw_err=90., min_yaw_err=60.0, freq_ctrl=200)
min_snap.sim.plot_result( \
    debug_array[0], flag_save=False, \
    save_dir="../trajectory/result", save_idx="0", \
    t_set=t_set_new, d_ordered=d_ordered)
print("########################################################")

########################################################
# Check whether the reference motor speed exceed the maximum motor speed
print("Sanity check (ref. motor speed) result : {}". \
    format(min_snap.sanity_check(t_set_new, d_ordered, d_ordered_yaw, flag_parallel=True)))
print("########################################################")

########################################################
# Trajectory tools - plot and save trajectory
traj_tool.plot_trajectory(t_set_new, d_ordered, d_ordered_yaw, 
    flag_save=False, save_dir='', save_idx='test')

# Save trajectory in yaml format (save time allocation and the coefficients of piece-wise polynomial trajectory)
traj_tool.save_trajectory_yaml(t_set, d_ordered, d_ordered_yaw, \
    traj_dir="../trajectory/", \
    traj_name="test")

# Save trajectory in csv format (save time allocation and the coefficients of piece-wise polynomial trajectory)
traj_tool.save_trajectory_csv(t_set, d_ordered, d_ordered_yaw, \
    traj_dir="../trajectory/", \
    traj_name="test", \
    freq=200)

# Plot 2D projection of trajectory
fig, axs = plt.subplots(1,1)
fig.set_size_inches((6,5))
traj_tool.plot_trajectory_2D_single(axs, t_set_new, d_ordered, d_ordered_yaw)
plt.show()

# Get the maximum speed of reference trajectory
traj_tool.get_max_speed(t_set_new, d_ordered, flag_print=True)

# Plot trajectory 3D-animation
traj_tool.plot_trajectory_animation( \
    t_set_new, d_ordered, d_ordered_yaw, \
    flag_save=False, save_dir='../trajectory', save_file='test')

print("########################################################")

########################################################
# Low fidelity check
alpha_sta = 1.00
t_set_sta, d_ordered, d_ordered_yaw = min_snap.update_traj_(points, t_set, alpha_sta*np.ones_like(t_set))
print("low fidelity evaluation: {}".\
      format(min_snap.sanity_check(t_set_sta, d_ordered, d_ordered_yaw, flag_parallel=True)))
t_set_debug = t_set_sta

########################################################
# High fidelity check
alpha_sim = 2.0
t_set_sim, d_ordered, d_ordered_yaw = min_snap.update_traj_(points, t_set, alpha_sim*np.ones_like(t_set))
print("high fidelity evaluation: {}".\
      format(min_snap.run_sim_loop(t_set_sim, d_ordered, d_ordered_yaw, flag_debug=True)))
t_set_debug = t_set_sim

print("########################################################")

########################################################
# Debug tools
traj_tool.plot_trajectory(t_set_debug, d_ordered, d_ordered_yaw, flag_save=False)
fig, axs = plt.subplots(1,1)
fig.set_size_inches((12,5))
traj_tool.plot_trajectory_2D_single(axs, t_set_debug, d_ordered, d_ordered_yaw)
plt.show()
traj_tool.get_max_speed(t_set_debug, d_ordered, flag_print=True)

# Plot optimization result

In [ ]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import os, sys, time, copy, argparse
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, '../')
from pyTrajectoryUtils.pyTrajectoryUtils.utils import *
from mfboTrajectory.utils import *
from mfboTrajectory.multiFidelityModelWaypoints import check_dataset_init
from mfboTrajectory.minSnapTrajectoryWaypoints import MinSnapTrajectoryWaypoints

traj_tool = TrajectoryTools(MAX_POLY_DEG = 9, MAX_SYS_DEG = 4, N_POINTS = 20)
min_snap = MinSnapTrajectoryWaypoints(MAX_POLY_DEG = 9, MAX_SYS_DEG = 4, N_POINTS = 20)

def get_min_time_array(filedir, filename, MAX_ITER=50):
    yamlFile = os.path.join(filedir, filename)
    min_time_array = []
    num_failure = 0
    with open(yamlFile, "r") as input_stream:
        yaml_in = yaml.load(input_stream)
        for i in range(MAX_ITER+1):
            if 'iter{}'.format(i) in yaml_in:
                min_time_array.append(np.float(yaml_in['iter{}'.format(i)]['min_time']))
                if 'exp_result' in yaml_in['iter{}'.format(i)]:
                    num_failure += 1-np.float(yaml_in['iter{}'.format(i)]['exp_result'])
    
    return min_time_array, num_failure

def get_snap_array(filedir, filename, points, t_set_sim, MAX_ITER=50):
    yamlFile = os.path.join(filedir, filename)
    min_time_array = []
    snap_array = []
    snap_min = 1.0
    num_failure = 0
    with open(yamlFile, "r") as input_stream:
        yaml_in = yaml.load(input_stream)
        for i in range(MAX_ITER+1):
            if 'iter{}'.format(i) in yaml_in:
                min_time_array.append(np.float(yaml_in['iter{}'.format(i)]['min_time']))
                if len(min_time_array) == 1 or min_time_array[-2] > min_time_array[-1]:
                    alpha_set = np.array(yaml_in['iter{}'.format(i)]['alpha_cand'])
                    print(t_set_sim)
                    _, _, _, snap = min_snap.update_traj(points, t_set_sim, alpha_set, \
                                                  yaw_mode=2, \
                                                  flag_run_sim=False, flag_return_snap=True)
                    snap_min = snap
                snap_array.append(snap_min)
                
                if 'exp_result' in yaml_in['iter{}'.format(i)]:
                    num_failure += 1-np.float(yaml_in['iter{}'.format(i)]['exp_result'])
    
    return min_time_array, num_failure, snap_array

sample_name = ['traj_1']
rand_seed = [123]

model_name = ['test_waypoints']

lb = 0.1
ub = 1.9
MAX_ITER = 5
fig = plt.figure(figsize=(10,8))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
for sample_name_ in sample_name[:]:
    points, t_set_sta = get_waypoints('../constraints_data/waypoints_constraints.yaml', 'traj_1', flag_t_set=True)
    t_dim = t_set_sta.shape[0]
    res_init, data_init = check_dataset_init(sample_name_, t_dim, N_L=1000, N_H=20, \
                                             lb=lb, ub=ub, sampling_mode=2, dataset_dir="../mfbo_data")
    if res_init:
        alpha_sim, X_L, Y_L, X_H, Y_H = data_init
        t_set_sim = t_set_sta*alpha_sim
    else:
        continue
    for model_idx_, model_name_ in enumerate(model_name[:]):
        test_data = np.empty((0,MAX_ITER+1))
        test_data_snap = np.empty((0,MAX_ITER+1))
        mean_failure = 0
        for rand_seed_ in rand_seed[:]:
            data, num_failure, data_snap = get_snap_array('../mfbo_data/{}'.format(sample_name_), \
                                      'result_{}_{}.yaml'.format(model_name_, rand_seed_), \
                                                         points, t_set_sim)
            mean_failure += num_failure
            if len(data) is not MAX_ITER+1:
                print("Incomplete result: {}_{}_{}".format(sample_name_,model_name_,rand_seed_))
            
            test_data = np.append(test_data, np.expand_dims(data,0), axis=0)
            test_data_snap = np.append(test_data_snap, np.expand_dims(data_snap,0), axis=0)
        
        test_data_mean = test_data.mean(axis=0)
        test_data_std = test_data.std(axis=0)
        test_data_snap_mean = test_data_snap.mean(axis=0)
        test_data_snap_std = test_data_snap.std(axis=0)
        mean_failure /= (1.0*len(rand_seed))
        print(mean_failure)
        ax1.plot(range(test_data_mean.shape[0]), test_data_mean, '-', label='{}'.format(sample_name_))
        ax1.fill_between(range(test_data_mean.shape[0]), \
                         test_data_mean-test_data_std, test_data_mean+test_data_std, \
                         alpha=0.2)
        ax2.plot(range(test_data_snap_mean.shape[0]), test_data_snap_mean, '-', label='{}'.format(sample_name_))
        ax2.fill_between(range(test_data_snap_mean.shape[0]), \
                         test_data_snap_mean-test_data_snap_std, test_data_snap_mean+test_data_snap_std, \
                         alpha=0.2)

ax1.set_title("Simulation result of waypoints trajectory", {'fontsize':20}, y=1.02)
ax1.set_ylabel("Relative flight time", {'fontsize':16})
ax1.set_xlabel("Iterations", {'fontsize':16})
ax1.tick_params(axis='both', which='major', labelsize=14)
ax2.set_ylabel("Relative smoothness", {'fontsize':16})
ax2.set_xlabel("Iterations", {'fontsize':16})
ax2.tick_params(axis='both', which='major', labelsize=14)
ax1.grid()
ax2.grid()
plt.tight_layout()
plt.show()
plt.pause(1e-6)